# Learning and Decision Making

## Laboratory 2: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the gridworld domain described in the Homework and which you modeled using a Markov decision process.

<img src="maze.png" width="200px">

Recall that:

* At each step, the agent may move in any of the four directions -- up, down, left and right. 
* Movement across a _grey_ cell division succeeds with a $0.8$ probability and fails with a $0.2$ probability. 
* Movements across colored cell divisions (blue or red) succeed with a $0.8$ probability _but only if the agent has the corresponding colored key_. Otherwise, they fail with probability $1$. 
* When the movement fails, the agent remains in the same cell. 
* To get a colored key, the agent simply needs to stand in the corresponding cell. 
* The goal of the agent is to reach the cell marked with **"G"**. 

**Throughout the lab, use $\gamma=0.99$.**

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs. Make sure that:
    * The costs lie in the interval [0, 1]
    * The cost for standing in goal cell is minimal
    * The cost for standing in intermediate cells is maximal

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [137]:
import numpy as np

gamma = 0.99

states = [(1,'both'),
          (2,'none'), (2,'red'), (2,'both'),
          (3,'none'), (3,'red'), (3,'both'),
          (4,'none'), (4,'red'), (4,'both'),
          (5,'none'), (5,'red'), (5,'both'),
          (6,'red'), (6,'both'), ('goal', 'both')]

def probs(possible_actions):
    probs = np.identity(16)
    
    for (orig, dest) in possible_actions:
        probs[states.index(orig)][states.index(orig)] = 0.2
        probs[states.index(orig)][states.index(dest)] = 0.8
    
    return probs


actions = ['up', 'down', 'left', 'right']

probs_u = probs([((4,'none'), (2,'none')),
                ((4,'red'), (2,'red')),
                ((4,'both'), (2,'both')),
                ((5,'none'), (3,'none')),
                ((5,'red'), (3,'red')),
                ((5,'both'), (3,'both')),
                ((6,'red'), (4,'red')),
                ((6,'both'), (4,'both'))])

probs_d = probs([((2,'none'), (4,'none')),
                ((2,'red'), (4,'red')),
                ((2,'both'), (4,'both')),
                ((3,'none'), (5,'none')),
                ((3,'red'), (5,'red')),
                ((3,'both'), (5,'both')),
                ((4,'none'), (6,'red')),
                ((4,'red'), (6,'red')),
                ((4,'both'), (6,'both'))])

probs_r = probs([((1,'both'), (2,'both')),
                ((2,'none'), (3,'none')),
                ((2,'red'), (3,'red')),
                ((2,'both'), (3,'both')),
                ((4,'none'), (5,'none')),
                ((4,'red'), (5,'red')),
                ((4,'both'), (5,'both')),
                ((5,'both'), ('goal','both'))])

probs_l = probs([((2,'red'), (1,'both')),
                ((2,'both'), (1,'both')),
                ((3,'none'), (2,'none')),
                ((3,'red'), (2,'red')),
                ((3,'both'), (2,'both')),
                ((5,'none'), (4,'none')),
                ((5,'red'), (4,'red')),
                ((5,'both'), (4,'both')),
                (('goal','both'), (5,'both'))])

probabilities = {'up': probs_u, 'down': probs_d, 'right': probs_r, 'left': probs_l}

costs = np.ones((len(states), len(actions)))
costs[-1] = [0,0,0,0]

### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $x$, always moves the agent to the cell closest to the goal (irrespectively of the number of keys in the agent's possession). If multiple such cells exist, the agent should select randomly between the two.

For example, suppose that the agent is in cell 2. It should then select randomly between the actions $D$ and $R$. Conversely, suppose that the agent is in cell 4. The knight should then select actions $R$ with probability 1.

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry $(x,a)$ has the probability of selecting action $a$ in state $x$.

---

In [82]:
def action_row(acs):
    row = np.zeros((len(actions)))
    for action in acs:
        row[actions.index(action)] = 1 / len(acs)
    
    return row


policy = np.array([action_row(['right']),
                  action_row(['right', 'down']),
                  action_row(['right', 'down']),
                  action_row(['right', 'down']),
                  action_row(['down']),
                  action_row(['down']),
                  action_row(['down']),
                  action_row(['right']),
                  action_row(['right']),
                  action_row(['right']),
                  action_row(['right']),
                  action_row(['right']),
                  action_row(['right']),
                  action_row(['up']),
                  action_row(['up']),
                  action_row(['right', 'up', 'down'])])

print(policy)

[[0.         0.         0.         1.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.5        0.         0.5       ]
 [0.         0.5        0.         0.5       ]
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         1.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.33333333 0.33333333 0.         0.33333333]]


---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2.

---

In [98]:

P_pi = np.zeros(probs_r.shape)

for action_i, action in enumerate(actions):
    P_pi += np.diag(policy[:, action_i]).dot(probabilities[action])

c_pi = np.zeros((len(states)))

for action_i, action in enumerate(actions):
    c_pi += np.diag(policy[:, action_i]).dot(costs[:, action_i])
    
J_pi = np.linalg.inv(np.identity(len(states)) - gamma * P_pi).dot(c_pi)

print(J_pi)

[  4.89502117 100.         100.           3.69420073 100.
 100.           2.47821842 100.         100.           2.47821842
 100.         100.           1.24688279 100.           3.69420073
   0.        ]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is _not_ optimal: use value iteration to compute $J^*$ and show that $J^*\neq J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [99]:
import time

eps = 10e-8

old_J = np.zeros(len(states))
err = 1
i = 0
t_start = time.time()
while err > eps:
    Qs = [None] * len(actions)
    for action_i, action in enumerate(actions):
        Qs[action_i] = costs[:, action_i] + gamma * probabilities[action].dot(old_J)
    
    new_J = np.min(Qs, axis=0)
    err = np.linalg.norm(new_J - old_J)
    i += 1
    old_J = new_J

t_end = time.time()

print(old_J)
print( "Equal!" if np.array_equal(old_J,J_pi) else "Different!")
print(i, "iterations")
print(t_end - t_start, "s")

[ 4.89502117 10.67823015  6.08086879  3.69420073 11.79196791  7.25193028
  2.47821842  9.55043002  7.25193028  2.47821842 10.67823015  8.40839
  1.24688279  8.40839     3.69420073  0.        ]
Different!
29 iterations
0.0030927658081054688 s


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [100]:
import time

eps = 10e-8

pi = np.ones((len(states), len(actions)))
quit = False
i = 0
t_start = time.time()
while not quit:
    cpi = np.zeros((len(states)))
    Ppi = np.zeros((probs_u.shape))
    for action_i, action in enumerate(actions):
        cpi += np.diag(pi[:, action_i]).dot(costs[:, action_i])
        Ppi += np.diag(pi[:, action_i]).dot(probabilities[action])
    
    J = np.linalg.inv(np.eye(len(states)) - gamma * Ppi).dot(cpi)
    
    Qs = [None] * len(actions)
    for action_i, action in enumerate(actions):
        Qs[action_i] = costs[:, action_i] + gamma * probabilities[action].dot(J)
    
    
    pinew = np.zeros((len(states), len(actions)))
    for action_i, action in enumerate(actions):
        pinew[:, action_i] = np.isclose(Qs[action_i], np.min(Qs, axis=0), atol=eps, rtol=eps).astype(int)
    
    pinew = pinew / np.sum(pinew, axis=1, keepdims=True)
    
    quit = np.isclose(pi, pinew, atol=eps, rtol=eps).all()
    pi = pinew
    i += 1

t_end = time.time()

print(pi)
print( "Equal!" if np.array_equal(policy,pi) else "Different!")
print(i, "iterations")
print(t_end - t_start, "s")

[[0.         0.         0.         1.        ]
 [0.         1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.5        0.5        0.        ]
 [0.         0.         1.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         1.         0.        ]
 [0.5        0.         0.5        0.        ]
 [0.         0.         0.         1.        ]
 [1.         0.         0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.33333333 0.33333333 0.         0.33333333]]
Different!
5 iterations
0.004802703857421875 s


### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Suppose that the agent is where depicted in Fig. 1, and consider the situations (i) where it has no keys; (ii) where it has only the red key; (iii) where it has both keys. For each of the three situations,  

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [160]:
def unit_dist(state):
    dist = np.zeros((len(states)))
    dist[states.index(state)] = 1
    return dist


for state_dist in [unit_dist((5, 'none')), unit_dist((5, 'red')), unit_dist((5, 'both'))]:
    run_costs = []
    for i in range(100):
        run_dist = state_dist
        cost = 0
        for k in range(10000):
            print(str(i), '-', str(k), end='\r', flush=True)
            action_i = np.random.choice(len(actions), 1, p = run_dist.dot(pi))[0]
            action = actions[action_i]
            state = np.random.choice(len(states), 1, p = run_dist.dot(probabilities[action]))[0]
            run_dist = unit_dist(states[state])
            cost += costs[state][action_i] * gamma ** k
        run_costs.append(cost)
    print(np.sum(run_costs) / 100)


3 - 5404201204 - 215 - 217- 223 - 224 - 229 - 251 - 270 - 276 - 328 - 366 - 371 - 439 - 487 - 500 - 520 - 538579 - 583 - 594 - 596 - 598 - 631 - 661 - 674 - 675 - 680 - 698 - 772 - 779 - 791 - 811 - 813 815 - 816 - 878 - 945 - 973 - 986 - 994 - 1016 - 1025 - 1088 - 1090 1173 - 1199 - 1220 - 1229 - 1238 - 1438 - 1501 - 1514 - 1531 - 1549 - 1628 - 1660 - 1703 - 1731 - 1763 - 1776 1790 - 1882 - 1915 - 1993 - 2024 - 2104 - 2114 - 2164 - 2212 - 2276 - 2286 - 2310 - 2396 - 2434 - 2437 - 2522 - 2540 - 2581- 2659 - 2664- 2770- 2881 - 2925 - 2964- 2992 - 3103 - 3202- 3214 - 3256 - 3291 - 3325 - 3436 - 3439 - 3470 - 3547 - 3658 - 3769 - 3829 - 3877 - 3880 - 3951 - 3991 - 4032 - 4078 - 4102 - 4213 - 4324 - 4435 - 4532 - 4546 - 4657 - 4768 - 4829 - 4879 - 4990 - 5101 - 5212 - 5323 - 5434 - 5545 - 5606 - 5656 - 5767 - 5878 - 5937 - 5989 - 6100 - 6115 - 6191 - 6211 - 6322 - 6433 - 6544 - 6655 - 6766 - 6877 - 6926 - 6988 - 7099 - 7210 - 7321 - 7432 - 7543 - 7654 - 7765 - 7876 - 7987 - 8031 - 8098 - 8

KeyboardInterrupt: 